# Open3D Guide: 8. ICP Registration

> This tutorial demonstrates the ICP (Iterative Closest Point) registration algorithm. It has been a mainstay of geometric registration in both research and industry for many years. The input are two point clouds and an initial transformation that roughly aligns the source point cloud to the target point cloud. The output is a refined transformation that tightly aligns the two point clouds. A helper function draw_registration_result visualizes the alignment during the registration process. In this tutorial, we show two ICP variants, the point-to-point ICP and the point-to-plane ICP [Rusinkiewicz2001].
>
> Both [ICP registration](https://www.open3d.org/docs/latest/tutorial/Advanced/global_registration.html) and [Colored point cloud registration](https://www.open3d.org/docs/latest/tutorial/Advanced/colored_pointcloud_registration.html) are known as **local registration methods** because they rely on a rough alignment as initialization. Prior to a local registration we need some kind of [**global registration**](https://www.open3d.org/docs/latest/tutorial/Advanced/global_registration.html). This family of algorithms do not require an alignment for initialization. They usually produce less tight alignment results and are used as initialization of the local methods.

**This notebook deals with the local registration approach ICP**: we give a source and target point cloud already aligned and we obtain a more tight alignment.

**IMPORTANT: The point-to-plane ICP algorithm uses point normals; we need to estimate them if they are not available**.

Sources: 

- ICP: [https://www.open3d.org/docs/latest/tutorial/Basic/icp_registration.html](https://www.open3d.org/docs/latest/tutorial/Basic/icp_registration.html).
- Global registrations: [https://www.open3d.org/docs/latest/tutorial/Advanced/global_registration.html](https://www.open3d.org/docs/latest/tutorial/Advanced/global_registration.html).
- Colored point cloud registrations: [https://www.open3d.org/docs/latest/tutorial/Advanced/colored_pointcloud_registration.html](https://www.open3d.org/docs/latest/tutorial/Advanced/colored_pointcloud_registration.html).

Summary of contents:

- Prepare Input Data: Source and Target
- Point-to-point ICP
  - `o3d.pipelines.registration.registration_icp`
  - `o3d.pipelines.registration.TransformationEstimationPointToPoint()`
- Point-to-plane ICP
  - `o3d.pipelines.registration.TransformationEstimationPointToPlane()`

In [1]:
import sys
import os
import copy

# Add the directory containing 'examples' to the Python path
notebook_directory = os.getcwd()
parent_directory = os.path.dirname(notebook_directory)  # Parent directory
sys.path.append(parent_directory)

In [2]:
import open3d as o3d
from examples import open3d_example as o3dex
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Prepare Input Data: Source and Target

In this section:

- I load a triangle mesh and obtain a point cloud from it
- Then, I create a copy of the point cloud which is shifted and rotated
  - The transformation is parametrized
  - I also compute the inverse with an error: this will be the initial alignment
- Then, both point clouds (source and target) are visualized
- Finally, the current registration error is evaluated

In [13]:
# Helper visualization function
def draw_registration_result(source, target, transformation):
    # Since the functions transform and paint_uniform_color change the point cloud,
    # we call copy.deepcopy to make copies and protect the original point clouds.
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [37]:
# Helper function to create a transformation matrix
def create_transformation_matrix(d, a_deg):
    a_rad = np.radians(a_deg)  # Convert angle from degrees to radians
    # Define the rotation matrix around the axis (1, 1, 1)
    R = o3d.geometry.get_rotation_matrix_from_axis_angle(np.array([1.0, 1.0, 1.0]) * a_rad)
    # Define the translation vector (d, d, d)
    T = np.array([d, d, d])
    # Create a homogeneous transformation matrix
    transform = np.eye(4)
    transform[:3, :3] = R
    transform[:3, 3] = T
    return transform

In [120]:
# Load the mesh from a PLY file
mesh = o3d.io.read_triangle_mesh("../models/monkey.ply")
mesh.compute_vertex_normals()

# Convert the mesh to a point cloud by sampling
pcd = mesh.sample_points_poisson_disk(number_of_points=2048)
pcd.estimate_normals()

In [121]:
# Calculate the AABB and its diagonal
aabb = pcd.get_axis_aligned_bounding_box()
aabb_diagonal_length = np.linalg.norm(aabb.get_extent())

# Set the translation distance to 10% of the AABB diagonal
d = 0.2 * aabb_diagonal_length
# Define the rotation angle (degrees)
a = 20

# Create the transformation matrix
transformation_matrix = create_transformation_matrix(d, a)
# Create the inverse transformation but going back only 50%, i.e., with an error
# This matrix would be like the initial transformation/alignment obtained from
# a global registration
transformation_matrix_aprox_inv = create_transformation_matrix(-1.0*0.5*d, -1.0*0.5*a)

# Apply the transformation to the point cloud to create a new point cloud
pcd_transformed = copy.deepcopy(pcd)
pcd_transformed = pcd_transformed.transform(transformation_matrix)

In [122]:
print(transformation_matrix)

[[ 0.88181977 -0.26909475  0.38727498  0.75099655]
 [ 0.38727498  0.88181977 -0.26909475  0.75099655]
 [-0.26909475  0.38727498  0.88181977  0.75099655]
 [ 0.          0.          0.          1.        ]]


In [123]:
print(transformation_matrix_aprox_inv)

[[ 0.96976953  0.18700199 -0.15677152 -0.37549828]
 [-0.15677152  0.96976953  0.18700199 -0.37549828]
 [ 0.18700199 -0.15677152  0.96976953 -0.37549828]
 [ 0.          0.          0.          1.        ]]


In [124]:
# Set source and target
source = pcd_transformed
target = pcd

In [125]:
# Visualize source and target
draw_registration_result(source, target, np.eye(4))

In [131]:
# Visuaalize source and target with the approximate alignment,
# i.e., after obtaining an initial transformation from global registration
threshold = 0.05*aabb_diagonal_length
print(threshold)
trans_init = transformation_matrix_aprox_inv
draw_registration_result(source, target, trans_init)

0.1877491386523795


In [132]:
# Evaluate initial alignment
# The function evaluate_registration calculates two main metrics:
# - fitness, which measures the overlapping area (# of inlier correspondences / # of points in target). The higher the better.
# - inlier_rmse, which measures the RMSE of all inlier correspondences. The lower the better.
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=3.789062e-01, inlier_rmse=1.066932e-01, and correspondence_set size of 776
Access transformation to get result.


## Point-to-point ICP

In general, the ICP algorithm iterates over two steps:

1. Find correspondence set `K={(p,q)}` from target point cloud P, and source point cloud Q transformed with current transformation matrix T
2. Update the transformation T by minimizing an objective function E(T) defined over the correspondence set K.

Different variants of ICP use different objective functions E(T): [BeslAndMcKay1992] [ChenAndMedioni1992] [Park2017].

We first show a point-to-point ICP algorithm [BeslAndMcKay1992] using the objective

    E(T) = sum(abs(p − T*q); (p,q) in K)

The class `TransformationEstimationPointToPoint` provides functions to compute the residuals and Jacobian matrices of the point-to-point ICP objective. The function `registration_icp` takes it as a parameter and runs point-to-point ICP to obtain the results.

In [134]:
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=9.980469e-01, inlier_rmse=5.909662e-02, and correspondence_set size of 2044
Access transformation to get result.
Transformation is:
[[ 0.94931697  0.26681441 -0.16615461 -0.77200859]
 [-0.19005553  0.90829013  0.37267672 -0.82042534]
 [ 0.25035211 -0.32220973  0.91296479 -0.61875355]
 [ 0.          0.          0.          1.        ]]


In [135]:
# The result multiplied by the transformation_matrix should yield the identity 
reg_p2p.transformation @ transformation_matrix

array([[ 0.98516835, -0.08452151,  0.14932993,  0.01652035],
       [ 0.08387797,  0.99641951,  0.01061385, -0.0011547 ],
       [-0.14969235,  0.00206906,  0.98873046,  0.01291504],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [137]:
# By default, registration_icp runs until convergence or reaches a maximum number of iterations (30 by default).
# It can be changed to allow more computation time and to improve the results further.
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=1.000000e+00, inlier_rmse=4.020841e-15, and correspondence_set size of 2048
Access transformation to get result.
Transformation is:
[[ 0.88181977  0.38727498 -0.26909475 -0.75099655]
 [-0.26909475  0.88181977  0.38727498 -0.75099655]
 [ 0.38727498 -0.26909475  0.88181977 -0.75099655]
 [ 0.          0.          0.          1.        ]]


In [138]:
# The result multiplied by the transformation_matrix should yield the identity 
reg_p2p.transformation @ transformation_matrix

array([[ 1.00000000e+00,  4.64382502e-16,  9.54172502e-16,
        -5.55111512e-16],
       [ 6.82637200e-16,  1.00000000e+00,  2.56102925e-16,
         8.88178420e-16],
       [ 5.85971036e-16,  2.06954757e-16,  1.00000000e+00,
        -4.44089210e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

## Point-to-plane ICP

The point-to-plane ICP algorithm [ChenAndMedioni1992] uses a different objective function

    E(T) = sum(dot((p − T*q), np)^2; (p,q) in K)

where np is the normal of point p.

[Rusinkiewicz2001] has shown that the point-to-plane ICP algorithm has a faster convergence speed than the point-to-point ICP algorithm.

`registration_icp` is called with a different parameter `TransformationEstimationPointToPlane`. Internally, this class implements functions to compute the residuals and Jacobian matrices of the point-to-plane ICP objective.

In [139]:
print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.497597e-16, and correspondence_set size of 2048
Access transformation to get result.
Transformation is:
[[ 0.88181977  0.38727498 -0.26909475 -0.75099655]
 [-0.26909475  0.88181977  0.38727498 -0.75099655]
 [ 0.38727498 -0.26909475  0.88181977 -0.75099655]
 [ 0.          0.          0.          1.        ]]
